# Croatian Sentiment Classification the old-fashioned way: 
## `Naive Bayes`, `Logistic Regression`, and `Ngrams`

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *
from fastai.data.all import *
import torch
from pathlib import Path
from fastdownload import download_url 
from fastai.text.data import *
from fastai.text.core import *
from fastai.text.all import *

In [2]:
data_path = Path("./data/")
lang = 'hr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
mdl_path = path/'models'
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
lm_fns = [os.getcwd()/mdl_path/lm_fns[0], os.getcwd()/mdl_path/lm_fns[1]]

In [4]:
from nlputils import split_wiki,get_wiki
docs_path = split_wiki(path, 'hr')
docs_path

data/hrwiki/docs already exists; not splitting


Path('data/hrwiki/docs')

In [5]:
(path/"filtered").exists()

True

In [18]:
train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
df = pd.concat([train_df,test_df], sort=False)
print(len(train_df), len(test_df))
test_df.head()

2836 708


,0,1
0,False,"Čekali smo konobara 15 min da bi napokon došao i rekao da nema gotovo ničega od dnevne ponude gableca, na što smo ga zamolili da donese jelovnik koji nije stigao niti nakon 10 min. Loša organizacija."
1,False,"Koktelima baš nisam bila zadovoljna, ali hrana jako dobra. Usluga bi mogla biti brža."
2,True,"Konoba Galija oduševljava svojim raznovrsnim jelovnikom u svako doba godine, a pritom ukusno pripremljenom hranom, savršenim desertima te ljubaznim osobljem..Moram izdvojiti predjelo Pjat s plodovima mora koje je, osim ukusno, predivno servirano za one koje cijene i estetiku. Naravno da je i u tom stilu i glavno jelo.Slavonka koja će se s obitelji uvijek vratiti ovoj konobi 🤗🤗"
3,False,Naručili smo tunu s blitvom i krumpirom.Tuna je bila dosta slana pa smo rekli osoblju.Bila je u umaku od soje.Jedina zamjerka bi bila kad smo tražili preporuku da nam je osoblje reklo da je u umaku od soje.To nam ne odgovora jer sa grila nam je tuna s maslin.uljem.Osoblje je bilo brzo i ljubazno.
4,True,Jedan od najboljih kebaba u gradu. Isplate se te dvije kune koliko je skuplji.


In [19]:
tok = SentencePieceTokenizer(special_toks=[], lang='hr')
dls = TextDataLoaders.from_df(df, valid_pct=0.2, 
                              is_lm=True, 
                              tok=tok,
                              bs=64, 
                              text_col=1, 
                              label_col=0,
                             )

In [16]:
print(f'\fThere are {len(dls.train.x)} movie reviews in the training set\n')
print(movie_reviews.train)

In [17]:
dls.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 ',',
 '.',
 'i',
 'je',
 'u',
 'hrana',
 'za',
 'se',
 '!',
 'osoblje',
 'na',
 'da',
 '…',
 'a',
 'sve',
 'su',
 'jako',
 'ljubazno',
 'od',
 'ne',
 'ali',
 'ambijent',
 'usluga',
 'super',
 'sam',
 'cijene',
 'odlična',
 'smo',
 'sa',
 'nije',
 'mjesto',
 's',
 'vrlo',
 '..',
 'ukusna',
 'kao',
 'to',
 'malo',
 'ugodan',
 'što',
 'dobra',
 'restoran',
 'bilo',
 'uvijek',
 'koji',
 'atmosfera',
 ')',
 '3',
 'bi',
 '(',
 '-',
 'pizza',
 'porcije',
 'samo',
 'više',
 'bila',
 'konobari',
 'm',
 'pohvale',
 'jer',
 'konobar',
 'mi',
 'pa',
 'hrane',
 'ugodno',
 'po',
 'uz',
 '"',
 'prostor',
 'ok',
 'bio',
 'lijepo',
 'klopa',
 'vrhunska',
 'nas',
 'dobar',
 'dobro',
 'te',
 'odlično',
 'jela',
 'ili',
 'odlicna',
 'kad',
 'brza',
 'preporuka',
 'odličan',
 'izvrsna',
 'ako',
 'ugodna',
 'nema',
 'hranu',
 'do',
 'koje',
 'fina',
 '5',
 'tako',
 'bez',
 'najbolji',
 'posluga',
 'svakako',
 'restora